## CÓDIGO DE OTIMIZAÇÃO PARA O PROJETO FISH
O código tem como objetivo definir quais os lotes à serem vendidos na semana de maneira que o lucro seja o maior possível.

### Função objetivo
A função objetivo atual é definida como **lucro = receita - custos**, onde:<br>
- **Receita**: Calculada como o produto do **preço do dia** pela **biomassa total** do lote, variando conforme o dia da semana determinado para a venda.<br>
- **Custos**: São a soma dos **custos totais** do lote e do **custo diário acumulado**, que aumenta proporcionalmente ao número de dias em que o lote é mantido antes de ser vendido.

### Regras / Restrições
As regras atuais definidas para a venda dos lotes são:
- Um lote pode ser vendido apenas uma vez
- O número máximo de lotes que podem ser vendidos num mesmo dia é **4**.
- Nenhuma venda é realizada sábado e domingo.

### Saída / Retorno

Quando executado, o código gera um arquivo **CSV** nomeado **`output.csv`**. Este arquivo contém os lotes selecionados para venda na semana, junto com informações detalhadas, como:  
- **Identificação do lote**  
- **Data da venda** (dia da semana escolhido)  
- **Custos totais**
- **Biomassa total do lote**  
- **Lucro gerado pela venda**  
- Outros **atributos relevantes para análise e tomada de decisão**  

Esse arquivo pode ser utilizado para análise operacional e otimização contínua.

### Importação de bibliotecas
Realiza a importação das bibliotecas. A biblioteca pandas é utilizada para manipulação de dados, leitura dos dados em CSV e geração de arquivo CSV com os resultados da otimização. São utilizados módulos da biblioteca ortools para os cálculos de otimização linear.

In [625]:
#Instalação do pacote de otimização, principalmente para google colab
#!pip install ortools

In [626]:
import pandas as pd
from datetime import datetime, timedelta
#Pacotes de otimização
from ortools.linear_solver import pywraplp
from ortools.linear_solver.pywraplp import Variable
#Pacote para data handling da otimização
from typing import Dict, Set, Tuple, Union, List, Any
from collections import defaultdict
#Biblioteca para formatação de moeda
from babel.numbers import format_currency
#Biblioteca para geração de valores aleatórios
import random  
#Teste de interface
from ipywidgets import widgets, interact

### Declaração de variáveis
Declaração e tipagem das variáveis que serão utilizadas no projeto.

In [627]:
#Criação de um Dictonary, que tem chave o n° do Lote e como valor outro Dictionary
lotes_info = {}
lotes_param = {}
venda_lote = {}
venda_lote_dia = {}
dias = range(7)
preco_dia = []
objective: str = "lucro"
solver = None
solverParams = None

### Configuração de solver
Solver ou resolvedor que é criado para encontrar a solução de problemas matemáticos complexos. Nessa configuração ele possui uma tolerância padrão do tipo float e valor de 0,001 e um tempo limite de execução de 60.000ms (60s).

In [628]:
def config_solver(gap_limit: float = 1e-3, time_limit: int = 60000 ):    
        global solver, solverParams    

        solver = pywraplp.Solver('hello_program', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)

        solver.EnableOutput()
        solver.SetTimeLimit(time_limit)

        solverParams = pywraplp.MPSolverParameters()
        solverParams.SetDoubleParam(solverParams.RELATIVE_MIP_GAP, gap_limit)

        return solver, solverParams

### Leitura de arquivos
É realizada a leitura dos arquivos CSV de dados e parâmetros e suas informações são convertidas para um DataFrame.

In [629]:
dados_lotes = pd.read_csv('dados_lote_fish.csv', delimiter=';')
param_otm = pd.read_csv('param_otm.csv', delimiter=';')

### Armazenamento de informações
Essa função percorre todas as linhas do DataFrame "todos_abatedouros" e armazena o conteúdo de cada linha nas suas respectivas variáveis.

In [630]:
def get_values():
    global lotes_info, dias, preco_dia

    for _, row in dados_lotes.iterrows():
        lote = str(row.id_lote)

        #Criação do Dictionary interno que contém as informações do lote
        if lote not in lotes_info:
            lotes_info[lote] = {}

        #Grava data de movimentação
        lotes_info[lote]["data_movimentacao"] = row.data_movimentacao
        #Grava o dia da semana de movimentação
        data_movimentacao = datetime.strptime(lotes_info[lote]["data_movimentacao"], "%Y-%m-%d %H:%M:%S")  
        lotes_info[lote]["dia_semana_movimentacao"] = data_movimentacao.weekday()
        #Gravação de outras informações do lote
        lotes_info[lote]["tanque_origem"] = row.tanque_origem
        lotes_info[lote]["especie"] = row.especie
        lotes_info[lote]["n_dias"] = row.n_dias
        lotes_info[lote]["custos_totais"] = row.custos_totais
        lotes_info[lote]["biomassa"] = row.biomassa
        lotes_info[lote]["biomassa_esperada_total"] = row.biomassa_esperada_total
        #Teste custo diário considerando custos totais e n° dias
        lotes_info[lote]["custo_diario"] = (row.custos_totais / row.n_dias)
        
    #Teste preço por dia
    #preco_dia = [round(random.uniform(7,7.5), 2) for _ in dias]
    preco_dia = [7.5, 7.39, 7.4, 7.2, 7.45, 7.6, 7.4]
    print("Preço diario:",preco_dia)

In [631]:
#Exibição dos lotes para teste
"""
global lotes_info

get_values()

import json
for lote in lotes_info:
    #print(f"Lote: {lote} {lotes_info[lote]}")
    print(json.dumps(lotes_info, indent=4))    
"""

'\nglobal lotes_info\n\nget_values()\n\nimport json\nfor lote in lotes_info:\n    #print(f"Lote: {lote} {lotes_info[lote]}")\n    print(json.dumps(lotes_info, indent=4))    \n'

### Parâmetros de cálculo
Nessa função são declarados os parâmetros para o calculo de otimização linear.

In [632]:
def get_params():
    lotes_param = {}

### Variáveis de otimização
A função configura variáveis para utilização no processo de otimização linear.

In [633]:
def set_variables_optimizer():
    global solver, lotes_info, venda_lote, venda_lote_dia, dias

    for lote in lotes_info:
        #Determina se o lote será ou não vendido
        venda_lote[lote] = solver.BoolVar(f'venda_lote_{lote}')
        for dia in dias:
            #Determina se o lote será ou não vendido no dia selecionado
            venda_lote_dia[lote, dia] = solver.BoolVar(f'venda_lote_{lote}_dia{dia}')

### Objetivo
Definição do objetivo da maximização do lucro.

In [634]:
def get_objective_value():
    global venda_lote, lotes_info, preco_dia, solver, dias, objective

    objective = solver.Objective()

    for lote in lotes_info:        
        for dia in dias:
            #Variáveis da função objetivo              
            receita = preco_dia[dia] * lotes_info[lote]["biomassa"]
            custo = lotes_info[lote]["custos_totais"] + (lotes_info[lote]["custo_diario"] * dia)
            #Função objetivo
            lucro = receita - custo         
            objective.SetCoefficient(venda_lote_dia[lote, dia], lucro)            
    objective.SetMaximization()

### Regras da otimização
A função define regras da otimização linear para garantir o melhor momento de venda do lote.

In [635]:
def optimization_rules():
    global solver, lotes_info, venda_lote, venda_lote_dia, dias

    
    for lote in lotes_info:
        #Garante que um lote seja vendido uma única vez
        solver.Add(sum(venda_lote_dia[lote, dia] for dia in dias) == venda_lote[lote])
        #Garante que um lote não tenha uma data de venda anterior à data de movimentação
        dia_minimo = lotes_info[lote]["dia_semana_movimentacao"]         
        solver.Add(sum(venda_lote_dia[lote, dia] for dia in range(dia_minimo - 1)) == 0)
        
    for dia in dias:
        #Máximo de 4 lotes por dia
        solver.Add(sum(venda_lote_dia[lote, dia] for lote in lotes_info) <= 4)
        #Nenhum lote é vendido sábado e domingo
        solver.Add(sum(venda_lote_dia[lote, 6] for lote in lotes_info) == 0)
        solver.Add(sum(venda_lote_dia[lote, 5] for lote in lotes_info) == 0)

### Execução do solver
A função realiza os cálculos de otimização linear

In [636]:
def solver_resolver():
    global solver, venda_lote, venda_lote_dia, solverParams, lotes_info, dias, preco_dia
    infos = []
    #Execução do solver
    status = solver.Solve(solverParams)
    sem = ("Segunda", "Terça", "Quarta", "Quinta", "Sexta", "Sábado", "Domingo")
    #Verificação de resultados ótimos ou viáveis
    if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
        print(f'Funcao objetivo ={solver.Objective().Value()}')
        print(solver.WallTime())
        total_lucro = 0
        for lote in lotes_info:
            for dia in dias:
                if venda_lote_dia[lote, dia].solution_value() == 1:
                    #Manipulação de data para definição do dia da venda
                    data_movimentacao = datetime.strptime(lotes_info[lote]["data_movimentacao"], "%Y-%m-%d %H:%M:%S")  
                    dia_semana = lotes_info[lote]["dia_semana_movimentacao"]                               
                    data_venda = data_movimentacao + timedelta(dia - dia_semana) 
                    #Exibição dos lotes selecionados
                    print(f'Lote {lote} será vendido em {data_venda}, {sem[dia]}.')                
                    lucro = round((preco_dia[dia] * lotes_info[lote]['biomassa']) - lotes_info[lote]['custos_totais'] - (lotes_info[lote]["custo_diario"] * dia),2)
                    #Armazenamento de informações do lote selecionado
                    infos.append([
                        lote,
                        data_venda,
                        lotes_info[lote]["tanque_origem"],
                        lotes_info[lote]["n_dias"],
                        lotes_info[lote]["especie"],
                        lotes_info[lote]["custo_diario"],
                        lotes_info[lote]["custos_totais"] + (lotes_info[lote]["custo_diario"]*dia),
                        lotes_info[lote]["biomassa"],
                        lotes_info[lote]["biomassa_esperada_total"],
                        lucro
                    ])
                    total_lucro += lucro        
        print(f'Lucro total: {format_currency(total_lucro, "BRL", locale="pt_BR")}')
        return infos
    else:
        print('Nao resolveu o problema!')
        return None

In [637]:
"""def show_infos(lote):
    lote_info = df[df["Lote"] == lote]
    display(lote_info.style.set_table_styles(
        [{'selector': 'th', 'props': [('font-weight', 'bold'), ('background-color', '#b352ff')]}]
    ))  """  

'def show_infos(lote):\n    lote_info = df[df["Lote"] == lote]\n    display(lote_info.style.set_table_styles(\n        [{\'selector\': \'th\', \'props\': [(\'font-weight\', \'bold\'), (\'background-color\', \'#b352ff\')]}]\n    ))  '

### Função: optmize
Esta função funciona por meio da chamada de funções variáveis, parâmetros e regras de otimização, através do solver faz o cálculo de otimização e gera o output.csv por meio da utilização da biblioteca pandas.

In [638]:
def optimize():

        config_solver()

        get_values()        
        get_params()
                
        set_variables_optimizer()
        get_objective_value()
        optimization_rules()        

        if not isinstance(f := solver_resolver(), type(None)):
            column_names = [
                "ID LOTE","DATA VENDA","TANQUE ORIGEM", "N DIAS", "ESPECIE", "CUSTO DIARIO", "CUSTOS TOTAIS", "BIOMASSA", "BIOMASSA ESPERADA TOTAL", "LUCRO"
            ]
            df = pd.DataFrame(f,columns=column_names) 
            df = df.sort_values(by="DATA VENDA")
            df.to_csv("output.csv", sep=";", decimal=",", columns=column_names, index=False)  

            def show_infos(lote):
                lote_info = df[df["ID LOTE"] == lote]
                display(lote_info.style.set_table_styles(
                    [{'selector': 'th', 'props': [('font-weight', 'bold'), ('background-color', '#003333'), ('color','white'), ('text-align','center'), ('font-size','12px')]},
                    {'selector': 'td', 'props': [('background-color', '#004c4c'), ('text-align','center'), ('color','white'), ('font-size','12px')]},
                    ]
                ))   

            # Criar dropdown com opções de lotes
            dropdown = widgets.Dropdown(
                options=df["ID LOTE"].tolist(),
                description='Selecione o Lote:',
                style = {"description_width":"120px"},
                layout = {"width":"200px"},          
            )
            
            interact(show_infos, lote = dropdown) 

### Execução do programa
Chamada da função Optimize, que realiza o processo de otimização e finzaliza gerando o arquivo CSV com os resultados.

In [639]:
optimize()

Preço diario: [7.5, 7.39, 7.4, 7.2, 7.45, 7.6, 7.4]
Funcao objetivo =222827.73993053965
14
Lote 5709 será vendido em 2022-01-06 00:00:00, Quinta.
Lote 6173 será vendido em 2022-01-03 00:00:00, Segunda.
Lote 6035 será vendido em 2022-01-07 00:00:00, Sexta.
Lote 5948 será vendido em 2022-01-07 00:00:00, Sexta.
Lote 5821 será vendido em 2022-01-07 00:00:00, Sexta.
Lote 5739 será vendido em 2022-01-07 00:00:00, Sexta.
Lote 6033 será vendido em 2022-01-03 00:00:00, Segunda.
Lote 6024 será vendido em 2022-01-05 00:00:00, Quarta.
Lote 6008 será vendido em 2022-01-05 00:00:00, Quarta.
Lote 5792 será vendido em 2022-01-06 00:00:00, Quinta.
Lote 6306 será vendido em 2022-01-04 00:00:00, Terça.
Lote 6236 será vendido em 2022-01-04 00:00:00, Terça.
Lote 6037 será vendido em 2022-01-05 00:00:00, Quarta.
Lote 6030 será vendido em 2022-01-04 00:00:00, Terça.
Lote 6206 será vendido em 2022-01-10 00:00:00, Segunda.
Lote 6185 será vendido em 2022-01-11 00:00:00, Terça.
Lote 6018 será vendido em 2022-01-

interactive(children=(Dropdown(description='Selecione o Lote:', layout=Layout(width='200px'), options=('6173',…

In [640]:
#RASCUNHOS E TESTES

# Dados fictícios do lote
data = {
    "Lote": ["Lote 1", "Lote 2", "Lote 3"],
    "Data da Venda": ["2025-02-03", "2025-01-29", "2025-02-01"],
    "Lucro": [1500.75, 2000.25, 1800.50],
    "Biomassa": [1200, 1400, 1300],
    "Custos Totais": [500, 600, 550]
}
df = pd.DataFrame(data)

# Função para exibir informações do lote
def exibir_informacoes(lote):
    lote_info = df[df["Lote"] == lote]
    display(lote_info.style.set_table_styles(
        [{'selector': 'th', 'props': [('font-weight', 'bold'), ('background-color', '#b352ff')]}]
    ))

# Criar dropdown com opções de lotes
dropdown = widgets.Dropdown(
    options=df["Lote"].tolist(),
    description='Lotes:',
    value="Lote 1"
)

# Interatividade
interact(exibir_informacoes, lote=dropdown)

interactive(children=(Dropdown(description='Lotes:', options=('Lote 1', 'Lote 2', 'Lote 3'), value='Lote 1'), …

<function __main__.exibir_informacoes(lote)>